### Implementing Data Validation using Protobuf in a Distributed System
**Description**: Use gRPC to implement a distributed system that validates messages using
Protobuf.

**Steps**:
1. Create a .proto file for gRPC service.
2. Implement server-side validation
    - Create a gRPC server
    - Bind the server to an address
    - Start server

In [2]:
pip install grpcio grpcio-tools protobuf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 18.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [grpcio-tools] [grpcio-tools]
Note: you may need to restart the kernel to use updated packages.


In [3]:
syntax = "proto3";

package validator;

message UserRequest {
  string name = 1;
  int32 age = 2;
  string email = 3;
}

message ValidationResponse {
  bool is_valid = 1;
  string error_message = 2;
}

service DataValidator {
  rpc ValidateUser(UserRequest) returns (ValidationResponse);
}

SyntaxError: invalid syntax (2125043051.py, line 3)

In [4]:
python -m grpc_tools.protoc --proto_path=. --python_out=. --grpc_python_out=. validator.proto

SyntaxError: invalid syntax (516342560.py, line 1)

In [5]:
import grpc
from concurrent import futures
import time
import re
from validator_pb2 import UserRequest, ValidationResponse
from validator_pb2_grpc import DataValidatorServicer, add_DataValidatorServicer_to_server

class DataValidatorService(DataValidatorServicer):
    def ValidateUser(self, request, context):
        is_valid = True
        error_message = ""

        if not request.name:
            is_valid = False
            error_message += "Name cannot be empty. "
        elif not re.match(r"^[a-zA-Z\s]+$", request.name):
            is_valid = False
            error_message += "Name must contain only letters and spaces. "

        if request.age < 0 or request.age > 150:
            is_valid = False
            error_message += "Age must be between 0 and 150. "

        if not re.match(r"[^@]+@[^@]+\.[^@]+", request.email):
            is_valid = False
            error_message += "Invalid email format. "

        return ValidationResponse(is_valid=is_valid, error_message=error_message)

def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    add_DataValidatorServicer_to_server(DataValidatorService(), server)
    server.add_insecure_port('[::]:50051')
    server.start()
    print("Server started on port 50051.")
    try:
        while True:
            time.sleep(86400)  # Keep the server running for a day
    except KeyboardInterrupt:
        server.stop(0)

if __name__ == '__main__':
    serve()

ModuleNotFoundError: No module named 'validator_pb2'

In [6]:
import grpc
from validator_pb2 import UserRequest
from validator_pb2_grpc import DataValidatorStub

def run_client():
    with grpc.insecure_channel('localhost:50051') as channel:
        stub = DataValidatorStub(channel)

        # Valid request
        user1 = UserRequest(name='John Doe', age=30, email='john.doe@example.com')
        response1 = stub.ValidateUser(user1)
        print(f"Validation for User 1: Is Valid = {response1.is_valid}, Error = '{response1.error_message}'")

        # Invalid request (empty name)
        user2 = UserRequest(name='', age=25, email='jane@example.com')
        response2 = stub.ValidateUser(user2)
        print(f"Validation for User 2: Is Valid = {response2.is_valid}, Error = '{response2.error_message}'")

        # Invalid request (invalid age)
        user3 = UserRequest(name='Peter Pan', age=200, email='peter@neverland.com')
        response3 = stub.ValidateUser(user3)
        print(f"Validation for User 3: Is Valid = {response3.is_valid}, Error = '{response3.error_message}'")

        # Invalid request (invalid email)
        user4 = UserRequest(name='Alice', age=28, email='alice.invalid')
        response4 = stub.ValidateUser(user4)
        print(f"Validation for User 4: Is Valid = {response4.is_valid}, Error = '{response4.error_message}'")

if __name__ == '__main__':
    run_client()

ModuleNotFoundError: No module named 'validator_pb2'